In [1]:
!pip install gradio pandas numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import gradio as gr
from collections import Counter
from sklearn.model_selection import train_test_split


In [3]:

url = "https://raw.githubusercontent.com/uzay00/KaVe-Egitim/master/ARGE/Sorular/1-%C4%B0seAl%C4%B1mRobotu/data.csv"
df = pd.read_csv(url)

df.columns = df.columns.str.strip().str.replace('\ufeff', '')

print("Kolonlar:", df.columns.tolist())
df.head()


Kolonlar: ['age', 'algorithm', 'gpa', 'social', 'y']


,age,algorithm,gpa,social,y
0,3.576113,3.728920,0.930603,4.438372,1
1,1.829891,4.393183,3.137649,4.831307,1
2,3.981593,4.050717,3.886304,4.865860,1
3,1.734623,4.605488,2.605315,3.663224,1
4,2.759050,4.288383,1.894925,3.238931,1


In [4]:
X = df[['age', 'algorithm', 'gpa', 'social']].values
y = df['y'].values


In [5]:
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

def knn_predict(X_train, y_train, x, k=3):
    distances = []
    for i in range(len(X_train)):
        dist = euclidean_distance(X_train[i], x)
        distances.append((dist, y_train[i]))

    distances.sort(key=lambda x: x[0])
    k_nearest = [label for (_, label) in distances[:k]]
    prediction = Counter(k_nearest).most_common(1)[0][0]
    return prediction


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
def evaluate_knn(X_train, y_train, X_test, y_test, k=3):
    correct = 0
    for i in range(len(X_test)):
        pred = knn_predict(X_train, y_train, X_test[i], k)
        if pred == y_test[i]:
            correct += 1
    return correct / len(X_test)

accuracy = evaluate_knn(X_train, y_train, X_test, y_test)
print(f"Model doğruluğu: {accuracy:.2f}")


Model doğruluğu: 1.00


In [8]:
def predict(age, algorithm, gpa, social):
    try:
        x = np.array([age, algorithm, gpa, social])
        prediction = knn_predict(X_train, y_train, x, k=3)
        return f"Önerilen Pozisyon: {prediction}"
    except Exception as e:
        return f"Hata: {str(e)}"


In [9]:
interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Number(label="Yaş"),
        gr.Number(label="Algoritma Skoru"),
        gr.Number(label="GPA"),
        gr.Number(label="Sosyal Beceriler Skoru")
    ],
    outputs="text",
    title=f"KNN ile İşe Alım Tahmini (Doğruluk: {accuracy:.2f})",
    description="Yaş, GPA, algoritma bilgisi ve sosyal beceriye göre pozisyon tahmini yapar."
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d75f360d431dfed235.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
